# Experiment 021: Systematic Tessellation Parameter Search

Based on the jiweiliu kernel, implement 2-tree tessellation with systematic parameter search.

Key parameters:
- 2-tree seed with angles ~75° and ~255°
- Translation distances a (x-spacing) and b (y-spacing)
- Grid dimensions (ncols, nrows)
- append_x and append_y flags

For each N, try multiple (ncols, nrows) combinations where 2*ncols*nrows >= N

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import json
from tqdm import tqdm

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def check_overlap(tree1, tree2):
    return tree1.overlaps(tree2) or tree1.contains(tree2) or tree2.contains(tree1)

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score_contribution(side, n):
    return (side ** 2) / n

print("Functions defined")

Functions defined


In [2]:
# 2-tree tessellation generator
# Based on jiweiliu kernel parameters

def generate_tessellation(n, seed_angle1=75, seed_angle2=255, a=0.87, b=0.75, ncols=None, nrows=None):
    """
    Generate n trees using 2-tree tessellation.
    
    seed_angle1, seed_angle2: angles of the two seed trees
    a: x-translation distance
    b: y-translation distance
    ncols, nrows: grid dimensions (if None, auto-calculate)
    """
    # Auto-calculate grid dimensions if not provided
    if ncols is None or nrows is None:
        # Find smallest grid that can hold n trees (2 trees per cell)
        total_cells = (n + 1) // 2
        ncols = int(np.ceil(np.sqrt(total_cells)))
        nrows = int(np.ceil(total_cells / ncols))
    
    # Seed tree positions (relative)
    seed1 = {'x': 0, 'y': 0, 'deg': seed_angle1}
    seed2 = {'x': 0.5 * a, 'y': 0.3 * b, 'deg': seed_angle2}  # Offset second seed
    
    trees_data = []
    
    # Generate grid
    for col in range(ncols):
        for row in range(nrows):
            if len(trees_data) >= n:
                break
            # First seed tree
            x1 = seed1['x'] + col * a
            y1 = seed1['y'] + row * b
            trees_data.append({'x': x1, 'y': y1, 'deg': seed1['deg']})
            
            if len(trees_data) >= n:
                break
            # Second seed tree
            x2 = seed2['x'] + col * a
            y2 = seed2['y'] + row * b
            trees_data.append({'x': x2, 'y': y2, 'deg': seed2['deg']})
    
    return trees_data[:n]

# Test
test_trees = generate_tessellation(10)
print(f"Generated {len(test_trees)} trees")
for t in test_trees[:4]:
    print(f"  x={t['x']:.3f}, y={t['y']:.3f}, deg={t['deg']:.1f}")

Generated 10 trees
  x=0.000, y=0.000, deg=75.0
  x=0.435, y=0.225, deg=255.0
  x=0.000, y=0.750, deg=75.0
  x=0.435, y=0.975, deg=255.0


In [3]:
# SA optimizer for tessellation
def sa_optimize_tessellation(trees_data, n, max_iter=5000, t_start=1.0, t_end=0.001):
    """Optimize tessellation configuration using SA."""
    positions = np.array([[t['x'], t['y'], t['deg']] for t in trees_data])
    
    def make_trees(pos):
        return [create_tree_polygon(p[0], p[1], p[2]) for p in pos]
    
    def has_any_overlap(trees):
        for i in range(len(trees)):
            for j in range(i+1, len(trees)):
                if check_overlap(trees[i], trees[j]):
                    return True
        return False
    
    def get_score(trees):
        side = get_bounding_box_side(trees)
        return calculate_score_contribution(side, n)
    
    # Initial state
    current_trees = make_trees(positions)
    
    # Check for initial overlaps and fix them
    scale = 1.0
    while has_any_overlap(current_trees) and scale < 5:
        scale *= 1.1
        scaled_pos = positions.copy()
        scaled_pos[:, 0] *= scale
        scaled_pos[:, 1] *= scale
        current_trees = make_trees(scaled_pos)
    
    if has_any_overlap(current_trees):
        return None, float('inf')
    
    if scale > 1:
        positions[:, 0] *= scale
        positions[:, 1] *= scale
    
    current_score = get_score(current_trees)
    best_positions = positions.copy()
    best_score = current_score
    
    cooling_rate = (t_end / t_start) ** (1 / max_iter)
    T = t_start
    pos_delta = 0.05
    angle_delta = 5
    
    for iteration in range(max_iter):
        idx = np.random.randint(n)
        old_pos = positions[idx].copy()
        
        # Random move
        move_type = np.random.choice(['pos', 'angle', 'both'])
        if move_type in ['pos', 'both']:
            positions[idx, 0] += np.random.uniform(-pos_delta, pos_delta)
            positions[idx, 1] += np.random.uniform(-pos_delta, pos_delta)
        if move_type in ['angle', 'both']:
            positions[idx, 2] = (positions[idx, 2] + np.random.uniform(-angle_delta, angle_delta)) % 360
        
        new_tree = create_tree_polygon(positions[idx, 0], positions[idx, 1], positions[idx, 2])
        
        # Check overlap
        has_overlap = False
        for j in range(n):
            if j != idx and check_overlap(new_tree, current_trees[j]):
                has_overlap = True
                break
        
        if has_overlap:
            positions[idx] = old_pos
        else:
            new_trees = current_trees.copy()
            new_trees[idx] = new_tree
            new_score = get_score(new_trees)
            
            delta = new_score - current_score
            if delta < 0 or np.random.random() < np.exp(-delta / T):
                current_trees = new_trees
                current_score = new_score
                if current_score < best_score:
                    best_score = current_score
                    best_positions = positions.copy()
            else:
                positions[idx] = old_pos
        
        T *= cooling_rate
    
    return best_positions, best_score

print("SA optimizer defined")

SA optimizer defined


In [4]:
# Load baseline for comparison
def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

best_df = pd.read_csv('/home/submission/submission.csv')
best_df['x'] = best_df['x'].apply(parse_s_value)
best_df['y'] = best_df['y'].apply(parse_s_value)
best_df['deg'] = best_df['deg'].apply(parse_s_value)
best_df['n'] = best_df['id'].apply(lambda x: int(x.split('_')[0]))

# Calculate baseline scores
baseline_scores = {}
for n in range(1, 201):
    group = best_df[best_df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    baseline_scores[n] = calculate_score_contribution(side, n)

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")

Baseline total: 70.630429


In [5]:
# Test tessellation on a few large N values
# Focus on N=100-200 where most score comes from

test_ns = [100, 144, 150, 196, 200]
results = {}

for n in test_ns:
    print(f"\n=== N={n} ===")
    print(f"Baseline: {baseline_scores[n]:.6f}")
    
    best_score = baseline_scores[n]
    best_config = None
    
    # Try different grid configurations
    # For 2-tree tessellation: 2 * ncols * nrows >= n
    min_cells = (n + 1) // 2
    
    configs_to_try = []
    for ncols in range(2, 20):
        for nrows in range(2, 20):
            if ncols * nrows >= min_cells and ncols * nrows <= min_cells + 10:
                configs_to_try.append((ncols, nrows))
    
    print(f"Trying {len(configs_to_try)} grid configurations...")
    
    for ncols, nrows in configs_to_try[:10]:  # Limit to 10 configs per N
        # Try different angle pairs
        angle_pairs = [
            (75, 255),   # jiweiliu default
            (60, 240),   # hexagonal-like
            (45, 225),   # diagonal
            (90, 270),   # perpendicular
        ]
        
        for angle1, angle2 in angle_pairs:
            trees_data = generate_tessellation(n, seed_angle1=angle1, seed_angle2=angle2, 
                                               a=0.87, b=0.75, ncols=ncols, nrows=nrows)
            optimized_pos, score = sa_optimize_tessellation(trees_data, n, max_iter=3000)
            
            if optimized_pos is not None and score < best_score:
                print(f"  [{ncols}x{nrows}] angles=({angle1},{angle2}): {score:.6f} - NEW BEST!")
                best_score = score
                best_config = (ncols, nrows, angle1, angle2, optimized_pos)
            elif optimized_pos is not None:
                pass  # Don't print non-improvements to reduce noise
    
    results[n] = {
        'baseline': baseline_scores[n],
        'best': best_score,
        'improvement': baseline_scores[n] - best_score,
        'config': best_config
    }
    print(f"Best for N={n}: {best_score:.6f} (improvement: {results[n]['improvement']:.6f})")


=== N=100 ===
Baseline: 0.343427
Trying 24 grid configurations...


Best for N=100: 0.343427 (improvement: 0.000000)

=== N=144 ===
Baseline: 0.342276
Trying 19 grid configurations...


Best for N=144: 0.342276 (improvement: 0.000000)

=== N=150 ===
Baseline: 0.337064
Trying 19 grid configurations...


Best for N=150: 0.337064 (improvement: 0.000000)

=== N=196 ===
Baseline: 0.333262
Trying 15 grid configurations...


Best for N=196: 0.333262 (improvement: 0.000000)

=== N=200 ===
Baseline: 0.337549
Trying 13 grid configurations...


Best for N=200: 0.337549 (improvement: 0.000000)


In [6]:
# Summary
print("\n" + "="*60)
print("TESSELLATION SEARCH SUMMARY")
print("="*60)

total_improvement = 0
for n, r in results.items():
    print(f"N={n}: baseline={r['baseline']:.6f}, best={r['best']:.6f}, improvement={r['improvement']:.6f}")
    total_improvement += r['improvement']

print(f"\nTotal improvement: {total_improvement:.6f}")
print(f"Baseline total: {sum(baseline_scores.values()):.6f}")
print(f"Target: 68.919154")


TESSELLATION SEARCH SUMMARY
N=100: baseline=0.343427, best=0.343427, improvement=0.000000
N=144: baseline=0.342276, best=0.342276, improvement=0.000000
N=150: baseline=0.337064, best=0.337064, improvement=0.000000
N=196: baseline=0.333262, best=0.333262, improvement=0.000000
N=200: baseline=0.337549, best=0.337549, improvement=0.000000

Total improvement: 0.000000
Baseline total: 70.630429
Target: 68.919154


In [7]:
# Save metrics
metrics = {
    'cv_score': sum(baseline_scores.values()),
    'improvements_found': len([n for n, r in results.items() if r['improvement'] > 1e-9]),
    'total_improvement': total_improvement,
    'results': {str(k): {'baseline': v['baseline'], 'best': v['best'], 'improvement': v['improvement']} 
                for k, v in results.items()}
}

with open('/home/code/experiments/021_tessellation_search/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved: {metrics}")

Metrics saved: {'cv_score': 70.63042896026437, 'improvements_found': 0, 'total_improvement': 0.0, 'results': {'100': {'baseline': 0.343427426569443, 'best': 0.343427426569443, 'improvement': 0.0}, '144': {'baseline': 0.34227612208517066, 'best': 0.34227612208517066, 'improvement': 0.0}, '150': {'baseline': 0.33706354699554264, 'best': 0.33706354699554264, 'improvement': 0.0}, '196': {'baseline': 0.3332622000853879, 'best': 0.3332622000853879, 'improvement': 0.0}, '200': {'baseline': 0.3375491782201624, 'best': 0.3375491782201624, 'improvement': 0.0}}}
